<a href="https://colab.research.google.com/github/YuriiKlim/AI/blob/learning_without_teacher/Lab2_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [2]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0
...,...,...,...
200135,6714,22,5.0
200136,6576,22,3.5
200137,6222,22,3.5
200138,6015,22,2.0


In [12]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [3]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["userId", "blog_id", "ratings"]], reader)

In [4]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [5]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [6]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [9]:
from surprise import KNNBasic

algo = KNNBasic(k=20,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [10]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [11]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")

Estimating biases using als...
mae = 1.6429312086900274
mse = 3.4498838105790863
rmse= 1.8573862846966127


In [13]:
algo.predict(uid=2356,
             iid=5994)

Prediction(uid=2356, iid=5994, r_ui=None, est=2.515759274387599, details={'was_impossible': False})

In [15]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.2 MB/s eta 0:00:00


In [22]:
def objective(trial):
    try:
        algorithm = trial.suggest_categorical('algorithm', ['BaselineOnly', 'SVD', 'KNNBasic', 'CoClustering'])

        if algorithm == 'BaselineOnly':
            method = trial.suggest_categorical('method', ['als', 'sgd'])
            reg_u = trial.suggest_float('reg_u', 1e-5, 1e-1, log=True)
            reg_i = trial.suggest_float('reg_i', 1e-5, 1e-1, log=True)
            bsl_options = {'method': method, 'reg_u': reg_u, 'reg_i': reg_i}
            algo = BaselineOnly(bsl_options=bsl_options)

        elif algorithm == 'SVD':
            n_factors = trial.suggest_int('n_factors', 20, 200)
            n_epochs = trial.suggest_int('n_epochs', 5, 50)
            lr_all = trial.suggest_float('lr_all', 1e-4, 1e-1, log=True)
            reg_all = trial.suggest_float('reg_all', 1e-4, 1e-1, log=True)
            algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

        elif algorithm == 'KNNBasic':
            k = trial.suggest_int('k', 10, 50)
            min_k = trial.suggest_int('min_k', 1, 5)
            sim_options = {
                'name': trial.suggest_categorical('name', ['cosine', 'pearson']),
                'user_based': trial.suggest_categorical('user_based', [True, False])
            }
            algo = KNNBasic(k=k, min_k=min_k, sim_options=sim_options)

        elif algorithm == 'CoClustering':
            n_cltr_u = trial.suggest_int('n_cltr_u', 3, 10)
            n_cltr_i = trial.suggest_int('n_cltr_i', 3, 10)
            n_epochs = trial.suggest_int('n_epochs', 20, 50)
            algo = CoClustering(n_cltr_u=n_cltr_u, n_cltr_i=n_cltr_i, n_epochs=n_epochs)

        algo.fit(trainset)

        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=False)

        return rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise optuna.exceptions.TrialPruned()

In [23]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2024-08-29 17:59:29,284] A new study created in memory with name: no-name-2741ca44-c803-4323-bf90-026f30d3389b


Computing the pearson similarity matrix...


[I 2024-08-29 17:59:34,620] Trial 0 finished with value: 1.939219846447997 and parameters: {'algorithm': 'KNNBasic', 'k': 11, 'min_k': 2, 'name': 'pearson', 'user_based': True}. Best is trial 0 with value: 1.939219846447997.
[I 2024-08-29 17:59:40,964] Trial 1 finished with value: 1.953098338260948 and parameters: {'algorithm': 'SVD', 'n_factors': 85, 'n_epochs': 32, 'lr_all': 0.017915447250944856, 'reg_all': 0.001520886663372155}. Best is trial 0 with value: 1.939219846447997.


Done computing similarity matrix.
Estimating biases using als...


[I 2024-08-29 17:59:41,944] Trial 2 finished with value: 1.8573167803671722 and parameters: {'algorithm': 'BaselineOnly', 'method': 'als', 'reg_u': 0.011062281205214228, 'reg_i': 2.0798243951937606e-05}. Best is trial 2 with value: 1.8573167803671722.
[I 2024-08-29 17:59:45,234] Trial 3 finished with value: 1.7774348534343387 and parameters: {'algorithm': 'SVD', 'n_factors': 189, 'n_epochs': 11, 'lr_all': 0.0009466574067987996, 'reg_all': 0.0008233154195149196}. Best is trial 3 with value: 1.7774348534343387.


...

[I 2024-08-29 18:04:04,656] Trial 75 finished with value: 2.013195729882197 and parameters: {'algorithm': 'KNNBasic', 'k': 22, 'min_k': 1, 'name': 'pearson', 'user_based': True}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:05,829] Trial 76 finished with value: 1.7688394193421229 and parameters: {'algorithm': 'SVD', 'n_factors': 20, 'n_epochs': 7, 'lr_all': 0.00019013099470235282, 'reg_all': 0.07682646918231852}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:07,589] Trial 77 finished with value: 1.7731592214305363 and parameters: {'algorithm': 'SVD', 'n_factors': 129, 'n_epochs': 7, 'lr_all': 0.0007381217926301452, 'reg_all': 0.0014980564982436742}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:15,546] Trial 78 finished with value: 2.0129364985170755 and parameters: {'algorithm': 'CoClustering', 'n_cltr_u': 5, 'n_cltr_i': 10, 'n_epochs': 21}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:17,309

Estimating biases using als...


[I 2024-08-29 18:04:18,343] Trial 80 finished with value: 1.8567210414286761 and parameters: {'algorithm': 'BaselineOnly', 'method': 'als', 'reg_u': 0.09802179716491273, 'reg_i': 0.004940435309216082}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:19,411] Trial 81 finished with value: 1.7691196607039152 and parameters: {'algorithm': 'SVD', 'n_factors': 25, 'n_epochs': 6, 'lr_all': 0.00019381932351044572, 'reg_all': 0.07393400721948772}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:20,353] Trial 82 finished with value: 1.769158873667812 and parameters: {'algorithm': 'SVD', 'n_factors': 26, 'n_epochs': 6, 'lr_all': 0.00018952433575318613, 'reg_all': 0.07556685685243872}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:21,834] Trial 83 finished with value: 1.76931898076892 and parameters: {'algorithm': 'SVD', 'n_factors': 34, 'n_epochs': 8, 'lr_all': 0.0003948697972090911, 'reg_all': 0.09994413827849839}. Best is trial 61

Computing the pearson similarity matrix...
Done computing similarity matrix.


[I 2024-08-29 18:04:34,907] Trial 90 finished with value: 1.8915638821625758 and parameters: {'algorithm': 'KNNBasic', 'k': 40, 'min_k': 3, 'name': 'pearson', 'user_based': True}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:36,309] Trial 91 finished with value: 1.7691680941491978 and parameters: {'algorithm': 'SVD', 'n_factors': 20, 'n_epochs': 5, 'lr_all': 0.0001076594351694058, 'reg_all': 0.06336902375128839}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:37,766] Trial 92 finished with value: 1.7693469631358907 and parameters: {'algorithm': 'SVD', 'n_factors': 25, 'n_epochs': 8, 'lr_all': 0.0001857209375419948, 'reg_all': 0.05077000838472365}. Best is trial 61 with value: 1.7686576507900638.
[I 2024-08-29 18:04:40,896] Trial 93 finished with value: 1.7694348227094945 and parameters: {'algorithm': 'SVD', 'n_factors': 33, 'n_epochs': 25, 'lr_all': 0.00013831196647052236, 'reg_all': 0.07727639607185216}. Best is trial 61 with value: 1.7686

In [24]:
print(f"Number of trials completed: {len(study.trials)}")

Number of trials completed: 100


In [25]:
study.best_params

{'algorithm': 'SVD',
 'n_factors': 25,
 'n_epochs': 5,
 'lr_all': 0.00015059845669634315,
 'reg_all': 0.0004337611599331474}

In [26]:
best_params = study.best_params

best_algorithm = best_params['algorithm']

if best_algorithm == 'BaselineOnly':
    bsl_options = {
        'method': best_params['method'],
        'reg_u': best_params['reg_u'],
        'reg_i': best_params['reg_i']
    }
    final_algo = BaselineOnly(bsl_options=bsl_options)

elif best_algorithm == 'SVD':
    final_algo = SVD(
        n_factors=best_params['n_factors'],
        n_epochs=best_params['n_epochs'],
        lr_all=best_params['lr_all'],
        reg_all=best_params['reg_all']
    )

elif best_algorithm == 'KNNBasic':
    sim_options = {
        'name': best_params['name'],
        'user_based': best_params['user_based']
    }
    final_algo = KNNBasic(
        k=best_params['k'],
        min_k=best_params['min_k'],
        sim_options=sim_options
    )

elif best_algorithm == 'CoClustering':
    final_algo = CoClustering(
        n_cltr_u=best_params['n_cltr_u'],
        n_cltr_i=best_params['n_cltr_i'],
        n_epochs=best_params['n_epochs']
    )

final_algo.fit(trainset)

final_predictions = final_algo.test(testset)

print(f"mae = {accuracy.mae(final_predictions, verbose=False)}")
print(f"mse = {accuracy.mse(final_predictions, verbose=False)}")
print(f"rmse= {accuracy.rmse(final_predictions, verbose=False)}")

mae = 1.6142757372467964
mse = 3.128772166176475
rmse= 1.7688335609029118
